### Recommendation Using TF-IDF weighted Word2Vec
Reference - [tfidf weighted word2vec](https://medium.com/analytics-vidhya/featurization-of-text-data-bow-tf-idf-avgw2v-tfidf-weighted-w2v-7a6c62e8b097)  , 
[中文版](https://renxingkai.github.io/2019/04/05/word-tfidf/?fbclid=IwAR3kWsXJq-SLMSqgUMG6y1ZSEUGtr_6MgLVr9USrUb981_3OjFqh7R_kMUs#TF-IDF%E5%8A%A0%E6%9D%83%E5%B9%B3%E5%9D%87%E8%AF%8D%E5%90%91%E9%87%8F) <br><br>
**STEP** <br>
1. Create TF-IDF 
2. Convert a tf-idf dictionary with word as key, idf as a value
3. Get TF-IDF features
4. Create Word2Vec Model
5. Combine w2v with TF-IDF
6. Calculate Cosine Similarity 
7. Recommend Law 

### Read ckiptagger & Dataframe

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import datetime

path = "./data"
ws = WS(path)

df = pd.read_csv('data_ETL3noPuncDict.csv')
# Replace '@' with ' ' in original dataframe
df.token = df.token.apply(lambda text: text.replace('@',' '))

### Tf-idf for Tokenized Text in Dataframe

In [2]:
# TF-IDF Model
tfidf_ml = TfidfVectorizer()
tfidf_ml.fit(df.token)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()

### Newly Entered Text Preprocess function
- Remove Punctuation
- Remove Spaces
- Sentence Segment
- turn into list

In [3]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    text = re.sub(' +', '',text)
    text = ws([text], sentence_segmentation=True)
    text = [x for l in text for x in l]
    return text

### Create Word2Vec Model

In [4]:
from gensim.models.word2vec import Word2Vec

w2v_model = Word2Vec(df.token.apply(lambda text: text.split()))
w2v_vocab = list(w2v_model.wv.vocab)
print(w2v_model)

Word2Vec(vocab=6182, size=100, alpha=0.025)


### Calculate TF-IDF Weighted Word2Vec

In [5]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in df.token.apply(lambda text: text.split()):
    text_vect = np.zeros(100)
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)

建立模型時間:  0:07:25.508385


### Law Recommendation Function
輸入內文 --> 跑出推薦的前十個相近內文對應的法律

In [6]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect):
    text = Preprocess(text)
    text_vect = np.zeros(100) # w2v size
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]
    tmp_top_10_law = df[['Ex_Tittle','CE_Item2','CE_Comment']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]]
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

### Try an Example
輸入內容便可以推薦出適合的法律<br>
(這邊列出的CE_Comment純粹是用來比對「輸入的內容」跟「原本內文」是否真的相近)

In [15]:
starttime = datetime.datetime.now()

newtext = '開標時有作拒絕往來廠商調查'
result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)

搜尋推薦時間:  0:00:01.262717


In [14]:
result.drop(columns = ['CE_Comment'])

,Ex_Tittle,CE_Item2,similarity_score
13476,政府採購法,103,96.4
11178,政府採購法,103,95.4
13171,政府採購法,103,95.2
2976,政府採購法,50,94.4
2388,政府採購法,50,92.4
13418,政府採購法,50,92.1
12739,政府採購法,50,92.1
9393,政府採購法,50,92.0
10749,政府採購法,50,91.4
673,政府採購法施行細則,55,91.1
